In [1]:
import types

In [2]:
async def async_function():
    return 1

In [3]:
async def async_generator():
    yield 1

In [4]:
type(async_function())

d:\python3.78\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: coroutine 'async_function' was never awaited
  """Entry point for launching an IPython kernel.


coroutine

In [5]:
type(async_generator())

async_generator

In [8]:
async_function().send(None)

StopIteration: 1

In [12]:
import os
import time
import sys

import requests

POP20_CC = ('CN IN US ID BR P1K NG BD RU JP MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = './downloads/'

def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)
        
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content

def show(text):
    print(text, end=' ')
    sys.stdout.flush()
    
def download_many(cc_list):
    for cc in sorted(cc_list):
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')
        
    return len(cc_list)

def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))
    
if __name__ == '__main__':
    main(download_many)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG P1K PH RU TR US VN 
20 flags downloaded in 21.82s


In [18]:
from concurrent import futures

from flags import save_flag, get_flag, show, main

MAX_WORKERS = 20

def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many(cc_list):
    # 设定工作的线程数：允许的最大值与要处理的数量之间的 最小值，以免创建多余的线程
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list))
        
    return len(list(res))

if __name__ == '__main__':
    main(download_many)

ConnectionError: HTTPConnectionPool(host='flupy.org', port=80): Max retries exceeded with url: /data/flags/bd/bd.gif (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001B7338DE608>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [19]:
def download_many(cc_list):
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))
            
        results = []
        for future in futures.as_completed(to_do):
            res = future.result()
            msg = '{} result: {!r}'
            print(msg.format(future, res))
            results.append(res)
            
    return len(results)